In [132]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import KBinsDiscretizer

## Initialization

In [133]:
static = pd.read_csv('../../data/raw/static.csv')
dynamic = pd.read_csv('../../data/raw/dynamic.csv')
notes = pd.read_csv('../../data/raw/notes.csv')

print('static')
display(static.head())
print('\ndynamic')
display(dynamic.head())
print('\nnotes')
display(notes.head())

static


,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,gender,race,admission_age,...,atrial_fibrillation,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke
0,24099382,6/11/11 7:15,6/18/11 21:45,6/11/11 14:31,6/13/11 18:05,2.15,0,M,OTHER,58.441631,...,0,0,0,0,0,1,1,1,0,0
1,27824879,9/30/28 20:55,10/10/28 15:54,9/30/28 20:58,10/5/28 18:39,4.90,0,F,WHITE,79.748598,...,0,0,0,0,0,1,0,1,1,0
2,27146619,9/20/41 0:34,9/22/41 12:05,9/20/41 7:13,9/21/41 18:10,1.46,0,F,WHITE,81.717397,...,0,0,1,1,0,0,0,0,1,0
3,22476441,10/8/73 16:46,10/16/73 13:05,10/8/73 18:28,10/10/73 13:26,1.79,0,F,BLACK/AFRICAN AMERICAN,63.769278,...,0,0,1,1,0,1,1,1,1,0
4,23169158,3/12/27 1:19,3/19/27 13:15,3/12/27 2:43,3/15/27 1:53,2.97,0,F,WHITE,84.191979,...,1,0,1,1,0,0,1,1,0,0



dynamic


,id,charttime,albumin,globulin,total_protein,aniongap,bicarbonate,bun,calcium,chloride,...,sodium_bg,lactate_bg,glucose_bg,d_dimer,fibrinogen,thrombin,inr,pt,ptt,urineoutput
0,28793466,4/12/29 3:35,NaN,NaN,NaN,16.0,21.0,12.0,8.8,106.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,10.9,27.8,NaN
1,25611175,11/29/40 3:50,NaN,NaN,NaN,15.0,27.0,28.0,8.5,101.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,26115624,9/7/50 0:22,3.5,NaN,NaN,12.0,22.0,9.0,7.9,111.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.2,13.9,25.8,NaN
3,28164589,3/11/59 1:11,NaN,NaN,NaN,12.0,34.0,58.0,8.3,103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.8,41.7,40.2,NaN
4,26115624,9/1/50 20:14,NaN,NaN,NaN,NaN,NaN,11.0,7.8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



notes


,note_id,id,note_type,note_seq,charttime,text
0,10002930-RR-19,25696644,RR,19,4/14/96 9:57,HISTORY: Altered mental status.\n\nTECHNIQUE:...
1,10003046-RR-7,26048429,RR,7,1/2/54 16:29,"INDICATION: Esophageal carcinoma, status post..."
2,10003046-RR-8,26048429,RR,8,1/3/54 7:33,HISTORY: Postop day one interval change.\n\nC...
3,10003400-RR-65,20214994,RR,65,2/24/37 16:01,EXAMINATION: CHEST (PRE-OP PA AND LAT)\n\nIND...
4,10003400-RR-66,20214994,RR,66,2/25/37 19:23,EXAMINATION: ABDOMINAL RADIOGRAPHS\n\nINDICAT...


In [134]:
dynamic.shape

(93604, 70)

In [135]:
static_unique_ids = static['id'].nunique()
dynamic_unique_ids = dynamic['id'].nunique()
notes_unique_ids = notes['id'].nunique()


print(f'static ids: {static_unique_ids}')
print(f'dynamic ids: {dynamic_unique_ids}')
print(f'notes ids: {notes_unique_ids}')

static ids: 20414
dynamic ids: 20414
notes ids: 20414


In [136]:
null_values_static = static.isna().sum().sum()
null_values_dynamic = dynamic.isna().sum().sum()
null_values_notes = notes.isna().sum().sum()

print(f'static null values: {null_values_static}')
print(f'dynamic values: {null_values_dynamic}')
print(f'notes values: {null_values_notes}')

static null values: 10026
dynamic values: 4543309
notes values: 0


In [137]:
null_values_static = static.isna().sum().sum()
null_values_dynamic = dynamic.isna().sum().sum()
null_values_notes = notes.isna().sum().sum()

datasets = ['Static', 'Dynamic', 'Notes']
null_counts = [null_values_static, null_values_dynamic, null_values_notes]

fig = go.Figure(
    data=[
        go.Bar(
        x=datasets, y=null_counts, 
        text=null_counts, textposition='outside', 
        textfont_size=16, texttemplate='%{text:,}', marker_color='#1F77B4')
        ]
    )

fig.update_layout(title='Null Values in Datasets',
                  xaxis_title='Datasets',
                  yaxis_title='Number of Null Values',
                  yaxis=dict(
                      showticklabels=True, 
                      showgrid=True
                      ),
                  xaxis=dict(
                      tickfont=dict(size=20),
                      showgrid=False
                      ),
                  width=1300,
                  height=800)

fig.show()

In [65]:
def na_ratio_acceptable(df, column_name, threshold=0.8):
    na_count = df[column_name].isna().sum()

    non_na_ratio = 1 - (na_count / df.shape[0])

    if non_na_ratio >= threshold:
        return True

    else:
        return False

In [66]:
def get_threshold_matrix(df):
    features = df.select_dtypes(include=[np.number]).columns.tolist()
    correlation_matrix = df[features].corr()

    threshold = 0.9
    threshold_matrix = correlation_matrix[abs(correlation_matrix) >= threshold]
    threshold_matrix = threshold_matrix.stack().reset_index().dropna(how='any')

    threshold_matrix.columns = ['variable_1', 'variable_2', 'correlation']
    threshold_matrix = threshold_matrix.sort_values(by='correlation', ascending=False)
    threshold_matrix = threshold_matrix[threshold_matrix.variable_1 != threshold_matrix.variable_2]

    threshold_matrix['sorted_pair'] = threshold_matrix.apply(lambda row: tuple(sorted([row['variable_1'], 
                                                                                    row['variable_2']])), 
                                                            axis=1)

    threshold_matrix = threshold_matrix.drop_duplicates(subset=['sorted_pair']).drop('sorted_pair', axis=1)

    return threshold_matrix


## Static Cleaning

In [67]:
static.shape

(20414, 25)

In [68]:
static_acceptable_cols = []

for col in static.columns:
    if na_ratio_acceptable(static, col):
        static_acceptable_cols.append(col)

static_acceptable_cols

['id',
 'hosp_admittime',
 'hosp_dischtime',
 'icu_intime',
 'icu_outtime',
 'los_icu',
 'icu_death',
 'gender',
 'race',
 'admission_age',
 'weight_admit',
 'admission_type',
 'first_careunit',
 'charlson_score',
 'atrial_fibrillation',
 'malignant_cancer',
 'chf',
 'ckd',
 'cld',
 'copd',
 'diabetes',
 'hypertension',
 'ihd',
 'stroke']

In [69]:
len(static_acceptable_cols)

24

In [70]:
x_values = ['before reduction', 'after reduction']
y_values = [len(static.columns), len(static_acceptable_cols)]      

# Create a bar chart
fig = go.Figure([go.Bar(x=x_values, y=y_values)])

# Customize layout
fig.update_layout(title='Before and After Column Reduction',
                  xaxis_title='Datasets',
                  yaxis_title='Column Count',
                  template='plotly_white')

# Show plot
fig.show()


In [71]:
static = static[static_acceptable_cols].dropna()
static_unique_ids = static['id'].nunique()
static['gender'] = static['gender'].map(lambda x: 1 if x == 'M' else 0)

print(f'static ids: {static_unique_ids}')

static ids: 19802


In [72]:
get_threshold_matrix(static)

,variable_1,variable_2,correlation


In [73]:
class BinStatic():

    def __init__(self, static):
        self.static = static

    def equalize_bins(self, static):
        los_icu=np.array(static['los_icu']).reshape(-1, 1)
        est = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')
        est.fit(los_icu)

        bin_edges = est.bin_edges_[0]

        print(bin_edges)

        max_value = los_icu.max()
        if bin_edges[-1] <= max_value:
            bin_edges[-1] = max_value + 0.1
            
        labels = ['1 to 2 days', '2 to 4 days', '4+ days']

        static['los_icu_binned'] = pd.cut(static['los_icu'], bins=bin_edges, labels=labels, right=False)

        return static

    def ohe(self, static_binned):
        ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        encoded = ohe.fit_transform(static_binned[['los_icu_binned']])
        ohe_cols = ohe.get_feature_names_out(['los_icu_binned'])
        ohe_los = pd.DataFrame(encoded, columns=ohe_cols)
        ohe_static = pd.concat([static, ohe_los], axis=1)

        return ohe_static

    def main(self):
        static_binned = self.equalize_bins(self.static)
        ohe_static = self.ohe(static_binned)

        return static_binned

In [74]:
binner = BinStatic(static)

static = binner.main()

static['los_icu_binned'].value_counts()

[  1.     2.03   4.13 101.73]


los_icu_binned
4+ days        6618
2 to 4 days    6602
1 to 2 days    6582
Name: count, dtype: int64

In [75]:
static.isna().sum()

id                     0
hosp_admittime         0
hosp_dischtime         0
icu_intime             0
icu_outtime            0
los_icu                0
icu_death              0
gender                 0
race                   0
admission_age          0
weight_admit           0
admission_type         0
first_careunit         0
charlson_score         0
atrial_fibrillation    0
malignant_cancer       0
chf                    0
ckd                    0
cld                    0
copd                   0
diabetes               0
hypertension           0
ihd                    0
stroke                 0
los_icu_binned         0
dtype: int64

## Dynamic Cleaning

In [76]:
dynamic_acceptable_cols = []

for col in dynamic.columns:
    if na_ratio_acceptable(dynamic, col):
        dynamic_acceptable_cols.append(col)

dynamic_acceptable_cols

['id',
 'charttime',
 'aniongap',
 'bicarbonate',
 'bun',
 'calcium',
 'chloride',
 'creatinine',
 'glucose',
 'sodium',
 'potassium']

In [77]:
dynamic = dynamic[dynamic_acceptable_cols].dropna()
dynamic = dynamic[dynamic['id'].isin(static['id'].unique())]
dynamic_unique_ids = dynamic['id'].nunique()

print(f'dynamic ids: {dynamic_unique_ids}')

dynamic ids: 19200


In [78]:
get_threshold_matrix(dynamic)

,variable_1,variable_2,correlation


In [79]:
thresholds = range(1, 11)
counts = []

for threshold in thresholds:
    filtered_id_counts = dynamic['id'].value_counts()[dynamic['id'].value_counts() >= threshold]
    counts.append(len(filtered_id_counts))

fig = go.Figure()

fig.add_trace(go.Scatter(x=list(thresholds), y=counts, mode='lines+markers'))

fig.update_layout(
    title='Number of Unique IDs by Minimum Number of Timesteps',
    xaxis_title='Minimum Timesteps',
    yaxis_title='Cohort Size',
    xaxis=dict(tickmode='array', tickvals=list(thresholds))
)

fig.show()

# fig.write_html('../data/figures/cohort-v-minimum-timesteps.html')

In [80]:
valid_ids = dynamic['id'].value_counts()[dynamic['id'].value_counts() >= 3].index.unique()

dynamic = dynamic[dynamic['id'].isin(valid_ids)]

print(f'dynamic shape: {dynamic.shape}')

print(f'cohort size: {dynamic["id"].nunique()}')

dynamic shape: (59452, 11)
cohort size: 9696


## Notes Cleaning

In [81]:
notes_acceptable_cols = []

for col in notes.columns:
    if na_ratio_acceptable(notes, col):
        notes_acceptable_cols.append(col)

notes_acceptable_cols

['note_id', 'id', 'note_type', 'note_seq', 'charttime', 'text']

In [82]:
notes = notes[notes_acceptable_cols].dropna()
notes_unique_ids = notes['id'].nunique()

print(f'notes ids : {notes_unique_ids}')

notes ids : 20414


In [83]:
notes = notes[notes['id'].isin(valid_ids)]

print(f'notes shape: {notes.shape}')

print(f'cohort size: {notes["id"].nunique()}')

notes shape: (47608, 6)
cohort size: 9696


In [84]:
def calculate_intervals(group):
    last_date = group['charttime'].max()
    group['interval'] = (last_date - group['charttime']).dt.days
    
    return group

In [85]:
notes['charttime'] = pd.to_datetime(notes['charttime'], format='%m/%d/%y %H:%M')
max_charttime = notes.groupby('id')['charttime'].transform('max')

notes['interval'] = (max_charttime - notes['charttime']).dt.days


notes = notes.sort_values(by=['id', 'charttime'], ascending=[False, True])

notes.head(20)

,note_id,id,note_type,note_seq,charttime,text,interval
68719,18434803-RR-10,29998399,RR,10,2012-05-08 18:52:00,INDICATION: ___ rear ended motor cycle into s...,1
68720,18434803-RR-11,29998399,RR,11,2012-05-08 19:09:00,EXAMINATION: CT HEAD W/O CONTRAST\n\nINDICATI...,0
68721,18434803-RR-12,29998399,RR,12,2012-05-08 19:09:00,EXAMINATION: CT C-SPINE W/O CONTRAST\n\nINDIC...,0
68722,18434803-RR-13,29998399,RR,13,2012-05-08 19:10:00,EXAMINATION: CT torso\n\nINDICATION: ___ rea...,0
68723,18434803-RR-14,29998399,RR,14,2012-05-08 19:12:00,EXAMINATION: SCROTAL U.S.\n\nINDICATION: ___...,0
68724,18434803-RR-15,29998399,RR,15,2012-05-08 20:21:00,INDICATION: ___ s/p MCC intubated// plz evalu...,0
68725,18434803-RR-16,29998399,RR,16,2012-05-08 20:41:00,"INDICATION: ___ year old man with open book, ...",0
68726,18434803-RR-17,29998399,RR,17,2012-05-08 21:02:00,INDICATION: ___ year old man with polytrauma ...,0
68727,18434803-RR-18,29998399,RR,18,2012-05-08 23:52:00,EXAMINATION: CT PELVIS WITHOUT CONTRAST\n\nIN...,0
68729,18434803-RR-20,29998399,RR,20,2012-05-09 04:01:00,EXAMINATION: CT HEAD W/O CONTRAST Q111 CT HEA...,0


In [86]:
static = static[static['id'].isin(valid_ids)]

# static.to_csv('../data/static_cleaned.csv', index=False)
# dynamic.to_csv('../data/dynamic_cleaned.csv', index=False)
# notes.to_csv('../data/notes_cleaned.csv', index=False)

In [87]:
static.id.nunique()

9696

In [88]:
dynamic.id.nunique()

9696

In [89]:
notes.id.nunique()

9696

In [90]:
static.shape

(9696, 25)

In [91]:
dynamic.shape

(59452, 11)

In [92]:
notes.shape

(47608, 7)

In [93]:
base_path = '../../data/split/with-outliers/combined/one-hot-encoded'

static_train = pd.read_csv(f'{base_path}/static_train.csv')
static_val = pd.read_csv(f'{base_path}/static_val.csv')
static_test = pd.read_csv(f'{base_path}/static_test.csv')

In [127]:
datasets = ['Total Cohort Size', 'Train', 'Val', 'Test']
null_counts = [len(static), len(static_train), len(static_val), len(static_test)]

fig = go.Figure(
    data=[
        go.Bar(
        x=datasets, y=null_counts, 
        text=null_counts, textposition='outside', 
        textfont_size=16, marker_color='#1F77B4'),
        ]
    )

fig.update_layout(title='Train Val Test Split',
                  xaxis_title='Datasets',
                  yaxis_title='Number Datapoints',
                  yaxis=dict(
                      showgrid=True
                      ),
                  xaxis=dict(
                      tickfont=dict(size=20),
                      ),
                  width=1300,
                  height=800)

fig.show()

In [124]:
color_map = {
    0: '#1F77B4', 
    1: 'rgb(82, 106, 131)'  
}

opacity_level = 1

fig = px.histogram(
    static, 
    x='los_icu', 
    color='icu_death', 
    nbins=50,  
    color_discrete_map=color_map,  
    marginal='rug',
    barmode='overlay',
    opacity=opacity_level,
    title='ICU Length of Stay Distribution by ICU Death'
)

fig.update_layout(
    title={'text': "ICU Length of Stay Distribution by ICU Death", 'x':0.5, 'xanchor': 'center'},
    xaxis_title='Length of Stay in ICU (days)',
    yaxis_title='Count',
    legend_title='ICU Death',
    # template='plotly_white',  
    width=1000, 
    height=600  
)

fig.update_layout(legend=dict(
    title_font_family="Times New Roman",
    font=dict(
        family="Courier",
        size=12,
        color="black"
    ),
    bgcolor="LightSteelBlue",
    bordercolor="Black",
    borderwidth=2
))

fig.show()


In [125]:
static

,id,hosp_admittime,hosp_dischtime,icu_intime,icu_outtime,los_icu,icu_death,gender,race,admission_age,...,malignant_cancer,chf,ckd,cld,copd,diabetes,hypertension,ihd,stroke,los_icu_binned
4,23169158,3/12/27 1:19,3/19/27 13:15,3/12/27 2:43,3/15/27 1:53,2.97,0,0,WHITE,84.191979,...,0,1,1,0,0,1,1,0,0,2 to 4 days
7,29179088,4/10/80 23:21,4/17/80 14:00,4/12/80 17:41,4/16/80 13:38,3.83,0,1,WHITE,72.276455,...,1,1,1,0,1,0,1,1,0,2 to 4 days
8,27139242,4/25/33 5:47,5/13/33 7:15,4/25/33 6:37,4/27/33 16:01,2.39,0,1,BLACK/AFRICAN AMERICAN,73.312782,...,1,0,0,0,0,0,0,0,0,2 to 4 days
9,25918892,10/27/43 15:06,10/31/43 14:21,10/27/43 15:06,10/30/43 12:11,2.88,0,1,WHITE,91.820358,...,0,0,0,0,0,0,1,1,1,2 to 4 days
10,20778724,10/4/67 7:15,10/19/67 17:45,10/5/67 9:23,10/6/67 21:43,1.51,0,1,ASIAN - CHINESE,63.756003,...,0,1,1,0,0,0,1,0,1,1 to 2 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20405,29961239,5/15/91 20:46,6/3/91 12:15,5/18/91 23:34,5/24/91 21:08,5.90,0,1,WHITE,74.370175,...,1,0,0,0,0,1,1,0,0,4+ days
20406,29962506,9/26/83 19:59,10/8/83 13:53,9/27/83 10:01,9/29/83 13:46,2.16,0,0,WHITE,79.734715,...,1,1,1,0,0,1,1,0,0,2 to 4 days
20407,29962832,7/25/63 17:07,8/7/63 16:00,7/25/63 17:08,8/3/63 18:42,9.07,0,1,BLACK/AFRICAN AMERICAN,77.563975,...,0,1,1,0,1,1,1,0,0,4+ days
20409,29968077,9/12/59 6:20,10/5/59 15:41,9/12/59 7:45,9/18/59 16:55,6.38,0,1,WHITE,73.695489,...,0,1,1,0,0,0,1,1,0,4+ days
